# Workflow for processing Lake George field data 03MAY2018

In [2]:
#import csv, glob, sys, os, re
#from os import listdir
import numpy as np
#import pandas as pd
#import math

import datacube
#from datetime import datetime, timedelta
#import pyproj
#from pandas import Series
#import matplotlib.pyplot as plt
import matplotlib

#import DEAPlotting
import calivali as cv
#
# Use notebook format (allows inline zooming and moving of figures)
# Set default font size for all plots
#
%matplotlib notebook
matplotlib.rcParams.update({'font.size': 11})

#
# Astropy is used to determine the Solar angle
#
import astropy.coordinates as coord
from astropy.time import Time
import astropy.units as u

#
# dc will be used later on when comparing with satellite data:
#
dc = datacube.Datacube(app='nbart-fieldsites')

Failed to resolve driver datacube.plugins.index::s3aio_index


### Set up input and output directories etc

In [ ]:
indir = '/g/data1a/u46/users/aw3463/GuyByrne/30APR18/lake_george/3_may_2018/'
output = '/g/data/u46/users/aw3463/GuyByrne/calval/PNGS/LAG-03MAY18/'

#
# field_data is in the format: Field Site Name, Date, Site number, Satellite Name
#
field_data = ['Lake George', '3/5/18', '', 'Landsat 8']

suffix = 'asd.rad.txt'

#
# panel_dir and in_panel refer to the file that is used in determining the k-factor,
# so this should remain unchanged.
#
panel_dir =  '/g/data1a/u46/users/aw3463/GuyByrne/30APR18/lake_george/26_march_2018/'
in_panel = 'ga_panel1_jan_2018.txt'
suffix2 = 'asd.ref.sco.txt'

#
# Setup for Landsat and Sentinel bandpass filters
#
sat_resp = dict({'Landsat 5': '/g/data1a/u46/users/aw3463/GuyByrne/misc_others/landsat5_vsir.flt', 
            'Landsat 7': '/g/data1a/u46/users/aw3463/GuyByrne/misc_others/landsat7_vsir.flt', 
            'Landsat 8': '/g/data1a/u46/users/aw3463/GuyByrne/misc_others/landsat8_vsir.flt',
            'Sentinel': '/g/data1a/u46/users/aw3463/GuyByrne/misc_others/sentinel2a_all.flt'})

f_name = sat_resp[field_data[3]]

#
# bad_pans    contains the names of individual panels readings that appear to be bogus
# bad_grounds contains ground-reading file names that appear to be bogus
# Use empty lists if there are no bad spectra.
#
#bad_grounds = ['site_200028.asd.rad', 'site_200029.asd.rad', 'site_2_line200030.asd.rad', 
#               'site_2_line200031.asd.rad', 'site_2_line400032.asd.rad', 'site_2_line400033.asd.rad',
#               'site_2_line800002.asd.rad', 'site_2_line200002.asd.rad', 'lkg_s2_ln10_00002.asd.rad']
bad_grounds = []
#bad_pans = []
bad_pans = ['lkg_line500038.asd.rad', 'lkg_line500039.asd.rad', 'lkg_line300000.asd.rad', 
            'lkg_line300001.asd.rad', 'lkg_line400000.asd.rad', 'lkg_line400001.asd.rad']


#
# Define the first line and spectrum number for all/good panels and grounds
#
firstGoodLine = 1
firstGoodPanelSpec = 0
firstGoodGroundSpec = 2

#
# Colours used for plotting Lines
#
colpac=['#770000', '#FF0000', '#FF7700', '#FFFF00', '#77FF00', '#00FF00', 
        '#00FF77', '#00FFFF', '#0077FF', '#0000FF', '#000077', '#FF00FF', '#777777', '#770077', '#777700']

### Define 'alldata' as the dataframe that contains all the raw spectra.
Show the first wavelength of every spectrum, so that the data integrity can be checked.

In [ ]:
alldata = cv.load_from_dir(indir, suffix)

alldata[alldata['Wavelength']==350]

In [ ]:
#
# Uncommenting the following line allows jupyter notebook to show more lines.
# This is needed when looking into the details of long dataframes/lists etc.
#
#pd.options.display.max_rows = 5000

### Specify which spectra are panels/ground/good/bad

 Determine panel file names by assuming that all panels have a data value of at least 0.06
 in the first wavelength (350nm). Call this dataframe 'panel_names'.

 good_panels = all panel data with bad panels removed<BR>
 bad_panels  = all bad panel data<BR>
 all_panels = both good and bad panel data<BR>
 good_grounds = good ground readings<BR>
 all_grounds = all ground data.<P>
 Any bad ground data (bad_grounds) is defined in the 2nd cell.


In [ ]:
panel_names, all_panels, good_panels, bad_panels, good_grounds, all_grounds = cv.extract_panels_grounds(alldata, bad_pans, bad_grounds)

### Create dataframes for all/good/bad panel spectra.

For the bad panel spectra dataframe, first check to see if bad panels have been defined.<BR>
If not, then don't create anything.

In [ ]:
all_panel_spec = cv.make_spec_df(all_panels)
good_panel_spec = cv.make_spec_df(good_panels)
try:
    bad_panel_spec = cv.make_spec_df(bad_panels)
except UnboundLocalError:
    pass

## Figure 1
### Plot panel radiances for all/good/bad panels

In [ ]:
cv.FIG_panel_radiances(good_panel_spec, bad_panel_spec, all_panel_spec, output, field_data)

# Figure 2

### Diagnosis plots of bad panel spectra

In [ ]:
good_panel_mean = cv.FIG_bad_panel_analysis(good_panel_spec, bad_panel_spec, field_data)

### Create spectral dataframes for all ground spectra and good ground spectra

In [ ]:
all_grounds_spec = cv.make_spec_df(all_grounds)
good_grounds_spec = cv.make_spec_df(good_grounds)

# Figure 3

### Plot ground spectra (all and good), normalised to the median good spectrum

These plots are used to identify any ground spectra that are bogus.

In [ ]:
cv.FIG_ground_spectra(good_grounds_spec, all_grounds_spec, field_data, output)

### Create time-relative dataframes

gpt = good panels<BR>
gpta = all panels<BR>
adt = good grounds<BR>
adta = all grounds

In [ ]:
gpt, gpta, adt, adta = cv.create_time_relative_dfs(good_panels, all_panels, good_grounds, all_grounds)

# Figure 4

### Plot timelines for ALL panel and ground data, with one line in one panel

In [ ]:
cv.FIG_all_timelines(gpta, adta, output, field_data)

# Figure 5

### Plot timelines for GOOD panel and ground data, with one line in one panel

In [ ]:
cv.FIG_good_timelines(gpta, gpt, adt, output, field_data)

# Figure 6

### Create timeline plot of averaged, normalised all/good panels

These plots are used to identify any panels that show unusually bright or dark readings,<BR>
    which can be weeded out as bad panels.
    
The general shape of the curve should follow "insolation" - the changing of incident light
due to the Sun rising/falling in the sky.

In [ ]:
gpt, gpta = cv.normalise_spectra(good_panel_mean, good_panel_spec, all_panel_spec, gpt, gpta)
    
cv.FIG_normalised_panels_timeline(gpt, gpta, output, field_data)

### Define the K-factor

This reads a standard file with a response curve for the detector, given an ideally white surface.<BR>
Then "k_f" is defined for the K-factor.

In [ ]:
k_f = cv.k_factor(panel_dir, in_panel)

### Rename the first spectrum in ALL/GOOD panels to the correct name

Rather than just "radiance", it will be named something like radiance1-0<BR>
for the zeroth spectrum in the first line, for example.    

In [ ]:
cv.spec_rename(good_panel_spec, good_grounds_spec, firstGoodLine, firstGoodPanelSpec, firstGoodGroundSpec)

### Create dataframe with Reflectances

In [ ]:
all_refls = cv.create_reflectances(good_panels, good_panel_spec, good_grounds_spec, k_f)

# Figure 7

### Plot all ground reflectances in black, plus the Line-averaged reflectances in colour

The Line-averaged reflectances are shown in order to identify any outlying lines that<BR>

might have been caused by bad panel spectra (for example).

In [ ]:
cv.FIG_reflectances(good_panels, all_refls, colpac, output, field_data)

### Apply weighted band responses to all reflectances

In [ ]:
result_df, band = cv.apply_weights(f_name, all_refls)

### Reformat band reflectances and apply to dataframe "ground_bands"

In [ ]:
ground_bands = cv.reformat_df(good_grounds, result_df)

# Figure 8

### Plot band reflectances

In [ ]:
cv.FIG_band_reflectances(ground_bands, result_df, band, colpac, output, field_data)

### Determine Solar angle

Based on the spectrum Latitude, Longitude and time stamp, calculate the angle of<BR>
the Sun, with respect to the zenith. Append this number to the "ground_bands" dataframe.

In [ ]:
def solar_angle(row):

    loc = coord.EarthLocation(lon=row['Longitude'] * u.deg,
                              lat=row['Latitude'] * u.deg)
    #timy0 = timei.to_pydatetime()
    timy = Time(row['date_saved'], format='datetime')
    
    altaz = coord.AltAz(location=loc, obstime=timy)
    sun = coord.get_sun(timy)

    return sun.transform_to(altaz).zen.degree

ground_bands['Solar_angle'] = ground_bands.apply(solar_angle, axis=1)

### Print out time stamp and coordinate extent

The time stamp and Lat/Long extents are required to calculate the BRDF correction, used below.

In order to calculate the BRF correction, the following method is used:

<OL>
    <LI>Run the print statements in the cell below.</LI>
    <LI>Copy and paste the output into a VDI terminal window</LI>
    <LI>Copy and paste the resultant VDI output into the BRDF calculation cell, writing over the similar text.</LI>
</OL>

NOTE: This works assuming that the BRDF code is in the directory /home/547/aw3463


In [ ]:
print("sed -i \"34s/.*/        setattr(self, 'acquisition_datetime', dateutil.parser.parse('",ground_bands['date_saved'][0],"'))/\" /home/547/aw3463/brdf/retrieve_brdf.py", sep='')
print("sed -i \"37s/.*/        bbox = geopandas.GeoDataFrame({'geometry': [box(",ground_bands['Longitude'].min(),", ", ground_bands['Latitude'].min(),", ",
      ground_bands['Longitude'].max(),", ", ground_bands['Latitude'].max(),")]})/\" /home/547/aw3463/brdf/retrieve_brdf.py", sep='')
print("python retrieve_brdf.py > temp.txt ; awk -f format.awk temp.txt")
print("")

### BRDF Calculation

The following script is adapted from FORTRAN code that will calculate the adjusted ASD measurement to 45$^\circ$. This is derived from "MODIS BRDF / Albedo Product: Algorithm Theoretical basis Docuement Version 5.0" by Strahler et al. (1999).

What follows is a key to the variables used in this code, compared to the variables and equations in the document:

hb and br = h/b and b/r . . . . . . . . . . . . . . . crown shape parameters, respectively.<BR>
RL_brdf = R($\theta, \vartheta, \phi, \Lambda$). . . . . . . . . . . . . . (37)<BR>
solar = $\theta$ . . . . . . . . . . . . . . . . . . . . . . . . . . solar zenith angle<BR>
view = $\xi$ . . . . . . . . . . . . . . . . . . . . . . . . . . .view zenith angle<BR>
ra = $\phi$ . . . . . . . . . . . . . . . . . . . . . . . . . . . . view-sun relative azimuth angle<BR>
cosxi = cos $\xi^\prime$ . . . . . . . . . . . . . . . . . . . . . . (43)<BR>
rs_thick = K$_{vol}$ = RossThick kernel . . . . . . (38)<BR>
d_li2 = D$^2$ . . . . . . . . . . . . . . . . . . . . . . . . . (42)<BR>
x_li = tan$\theta^\prime$ tan$\vartheta^\prime$ sin$\phi$ . . . . . . . . . . . . . . .(41) (part of)<BR>
cosl = cos $t$ . . . . . . . . . . . . . . . . . . . . . . . . (41)<BR>
l_li = $t$ . . . . . . . . . . . . . . . . . . . . . . . . . . . . .constrained such that $-1 \leq t \leq 1$<BR>
o_li = O($\theta, \vartheta, \phi$) . . . . . . . . . . . . . . . . . . . .(40)<BR>
li_sparse = K$_{geo}$ . . . . . . . . . . . . . . . . . . . . (39)<BR><BR><BR>

Equations used for the Ross-Li BRDF model:

R($\theta, \vartheta, \phi, \Lambda$) = $f_{iso}(\Lambda) + f_{vol}(\Lambda)\,K_{vol}(\theta, \vartheta, \phi) + f_{geo}(\Lambda)\,K_{geo}(\theta, \vartheta, \phi)$ . . . . . . . . (37)

$K_{vol} = k_{RT} = \frac{(\pi/2 - \xi)\rm{cos}\,\xi + \rm{sin}\,\xi}{\rm{cos}\,\theta + \rm{cos}\,\vartheta} - \frac{\pi}{4}$ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . (38)
    
$K_{geo} = k_{LSR} = O(\theta, \vartheta, \phi) - {\rm sec}\,\theta^\prime - {\rm sec}\,\vartheta^\prime + \frac{1}{2} (1 + \rm{cos}\,\xi^\prime)\, \rm{sec}\,\theta^\prime \rm{sec}\,\vartheta^\prime$ . . . (39)

$O = \frac{1}{\pi}(t - \rm{sin}\,t\,\,\rm{cos}\,t)(\rm{sec}\,\theta^\prime + \rm{sec}\,\vartheta^\prime)$ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . (40)

$\rm{cos}\,t = \frac{h}{b}\frac{\sqrt{D^2 + (\rm{tan}\,\theta^\prime\,\,\rm{tan}\,\vartheta^\prime\,\,\rm{sin}\,\phi)^2}}{\rm{sec}\,\theta^\prime + \rm{sec}\,\vartheta^\prime}$ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . (41)

$D = \sqrt{\rm{tan}^2\,\theta^\prime + \rm{tan}^2\,\vartheta^\prime - 2\,\rm{tan}\,\theta^\prime\,\rm{tan}\,\vartheta^\prime\,\rm{cos}\,\phi}$ . . . . . . . . . . . . . . . . . . . . . . . . . . (42)

$\rm{cos}\,\xi^\prime = \rm{cos}\,\theta^\prime\,\rm{cos}\,\vartheta^\prime + \rm{sin}\,\theta^\prime\,\rm{sin}\,\vartheta^\prime\,\rm{cos}\,\phi$ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .(43)

$\theta^\prime = \rm{tan}^{-1}(\frac{b}{r}\rm{tan}\,\theta)\,\vartheta^\prime = \rm{tan}^{-1}(\frac{b}{r}\rm{tan}\,\vartheta)$ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .(44)

In [ ]:
brdf_data = np.array([['', 'brdf0', 'brdf1', 'brdf2'],
                      ['band1', 0.08009322916666667, 0.020404166666666668, 0.012672395833333334],
                      ['band2', 0.08009322916666667, 0.020404166666666668, 0.012672395833333334],
                      ['band3', 0.11958697916666668, 0.040722395833333334, 0.019657291666666667],
                      ['band4', 0.13484114583333334, 0.04903385416666667, 0.02112291666666667],
                      ['band5', 0.25205, 0.14025364583333333, 0.026425],
                      ['band6', 0.32047552083333336, 0.16920885416666667, 0.03346458333333333],
                      ['band7', 0.23522083333333335, 0.08867760416666667, 0.033071875],
                     ])

ground_brdf, hb, br = cv.ReadAndCalc(brdf_data, ground_bands)

### Choose between Landsat 8 and Sentinel satellite data

In the last two lines, swap "ls8_bands" and "senti_bands" for the appropriate satellite overpass.

In [ ]:
band_nn = cv.get_spectrum_curve(f_name)
senti_bands = [1,2,3,4,5,6,7,8,'8a',9,11,12]
ls8_bands = [1,2,3,4,5,6,7,8]

band_min = [band_nn['band'+str(i)][:, 0].min() for i in ls8_bands]
band_max = [band_nn['band'+str(i)][:, 0].max() for i in ls8_bands]


# Figure 9

### Plot satellite band extents against median ground spectrum

This plot will show where the satellite bands fall, with respect to the spectrum<BR>
and in particular, with respect to the atmospheric absorbtion features.

In [ ]:
cv.FIG_band_extents(all_refls, band_min, band_max, output, field_data)

### Print field data time and Latitude/Longitude extents

The print outputs are used to manually feed the following query cell the right information<BR>
for the field data. Depending on how the field data align with satellite data, the longitude<BR>
and latitude extents may need to be adjusted.

In [ ]:
print(ground_bands['date_saved'][0])
print(ground_bands['Latitude'].min(), ground_bands['Latitude'].max(),
      ground_bands['Longitude'].min(), ground_bands['Longitude'].max())


### Query Satellite data

Retrieve xarrays for satellite data, based on query area and time.<BR>
sat_array will contain all band data<BR>
sat_rgb only contains the red, green and blue bands.

In [ ]:
query = {
         'time': ('2018-05-02', '2018-05-04'),
         'lat': (-35.0930, -35.0921),
         'lon': (149.4613, 149.4627),
         'output_crs': 'EPSG:3577',
         'resolution': (-25, 25),
        }

query2 = {
         'time': ('2018-05-02', '2018-05-04'),
         'lat': (-35.1030, -35.0821),
         'lon': (149.4513, 149.4727),
         'output_crs': 'EPSG:3577',
         'resolution': (-25, 25),
        }

#dc2 = datacube.Datacube(config='/home/547/aw3463/test_LS8.conf')

sat_array, sat_bigarray = cv.create_sat_arrays(dc, query, query2)

# Figure 10

### Plot relative locations of field and satellite data

In [ ]:
xloc = cv.FIG_sat_field_locations(ground_brdf, sat_array, colpac, output, field_data)

### Create Australian albers columns for ground_brdf (not used)

In [ ]:
#for i in range(len(ground_brdf)):
#    ground_brdf['Xalbers'], ground_brdf['Yalbers'] = pyproj.transform(wgs_84, aus_albers, ground_brdf['Longitude'][i], ground_brdf['Latitude'][i])
#    
#print(ground_brdf['Xalbers'][4], ground_brdf['Yalbers'][4])
#
#pyproj.transform(wgs_84, aus_albers, ground_brdf['Longitude'][4], ground_brdf['Latitude'][4])

### Create Field full band xarray

The field xarray is based on the pixel locations of the satellite data, where each pixel<BR>
    contains an average of all field data measurements that fall within the pixel.

In [ ]:
field_array = cv.create_field_from_sat(sat_array, ground_brdf, xloc)

# Figure 11

### Plot large-area context RGB array for Satellite data

In [ ]:
cv.FIG_sat_bigRGB(sat_bigarray, output, field_data)

# Figure 12

### Plot RGB array for Satellite data

In [ ]:
cv.FIG_sat_RGB(sat_array, output, field_data)

# Figure 13

### Plot RGB array for Field data

In [ ]:
cv.FIG_field_RGB(field_array, output, field_data)

### Test query for Sentinel 2 data (not used)

In [ ]:
# dc2 = datacube.Datacube(config='/home/547/aw3463/sentinel2.config')
#
#query = {
#        'lat': (-35.25, -35.35),
#        'lon': (149.05, 149.17),
#        'output_crs': 'EPSG:3577',
#        'resolution': (-10, 10),
#        'time':('2017-10-01', '2018-05-15')
#        }
#
#canberra = dc2.load(product='s2b_ard_granule', **query)
#
#canberra

# Figure 14

### Plot ratio arrays for each band

Each panel shows the ratio of satellite/field data.

In [ ]:
cv.FIG_ratio_arrays(sat_array, field_array, output, field_data)

### Create a statistics dataframe, comparing satellite and field data

In [ ]:
fstat_df = cv.create_stats(sat_array, ground_brdf)

### Convert all statistics entries to float

# Figure 15

### Plot comparison spectra of ALL satellite and field data, on a pixel-by-pixel basis

Error bars are shown for the field data, based on the standard deviation of the pixels<BR>
    within the field.

In [ ]:
cv.FIG_ALL_sat_field_bands(fstat_df, output, field_data)

# Figure 16

### Plot comparison spectra of INNER satellite and field data, on a pixel-by-pixel basis

Error bars are shown for the field data, based on the standard deviation of the pixels
within the field.

Only inner pixels are chosen to compare, where there are many field spectra for each satellite<BR>
    pixel. For example, using [2:4,2:4] will choose four pixels between coordinates (2,2) and (3,3),<BR>
    inclusive, from the top-left corner.

In [ ]:
inpix = [2, 4, 2, 4]
finner_df = cv.create_SUB_stats(sat_array, field_array, fstat_df, inpix)

In [ ]:
cv.FIG_SUB_sat_field_bands(finner_df, output, field_data)

# Figure 17

### Comparison plot of Field and satellite data

Plot shows a pixel-by-pixel comparison of all pixels where field data exists.<BR> 
Different band data are shown in different colours and different symbols.

In [ ]:
cv.FIG_sat_field_scatter_compare(sat_array, field_array, output, field_data)